This code is to solve the PDE which contains nonlinear operator by iteration method and Transnet

In [2]:
# kovasznay_transnet_picard_strict_div_cover_circle.py
# Strict TransNet (no s0_m):
#   φ_m(x) = tanh( γ * ( a_m^T (x - x_c) + r_m ) ), |a_m|=1,
#   a_m ~ Unif(unit circle), r_m ~ Unif([0, R]), with circle (x_c, R) slightly covering Ω.
# Picard iteration for steady NS (Kovasznay) with:
#   momentum residual, divergence residual, Dirichlet BC, one-point pressure pin.
# Golden-section search over γ_u and γ_p minimizing η(γ)=min_w ||A(γ)w - b||^2.

import numpy as np
import math, os
import matplotlib.pyplot as plt

# ----------------------- Problem (Kovasznay) -----------------------
pi = math.pi
Re = 40.0
nu = 1.0 / Re
lam = 0.5/nu - math.sqrt(1.0/(4*nu*nu) + 4*pi*pi)

# domain
x_min, x_max = -0.5, 1.0
y_min, y_max = -0.5, 1.5

def u_exact(XY):
    x = XY[:,0]; y = XY[:,1]
    ex = np.exp(lam * x)
    u1 = 1.0 - ex * np.cos(2*pi*y)
    u2 = (lam/(2*pi)) * ex * np.sin(2*pi*y)
    return np.stack([u1,u2], axis=1)

def p_exact(XY):
    x = XY[:,0]
    return 0.5*(1.0 - np.exp(2*lam*x))

# ----------------------- Sampling -----------------------
def uniform_grid_2d(nx, ny, x0,x1,y0,y1):
    xs = np.linspace(x0,x1,nx)
    ys = np.linspace(y0,y1,ny)
    XX,YY = np.meshgrid(xs,ys,indexing='xy')
    return np.stack([XX.ravel(), YY.ravel()], axis=1), xs, ys

def boundary_points_2d(n_per_side, x0,x1,y0,y1):
    xs = np.linspace(x0,x1,n_per_side)
    ys = np.linspace(y0,y1,n_per_side)
    return np.concatenate([
        np.stack([xs, np.full_like(xs,y0)],axis=1),
        np.stack([xs, np.full_like(xs,y1)],axis=1),
        np.stack([np.full_like(ys,x0), ys],axis=1),
        np.stack([np.full_like(ys,x1), ys],axis=1),
    ], axis=0)

def random_in_box(n, x0,x1,y0,y1, rng):
    return np.stack([rng.uniform(x0,x1,size=n), rng.uniform(y0,y1,size=n)], axis=1)

# -------------------- Strict TransNet Features (global xc,R) --------------------
class RandFeatStrict:   #参数化的特征函数工厂
    r"""
    φ_m(z) = tanh( γ * ( a_m^T (z - x_c) + r_m ) ),
    with |a_m|=1, a_m ~ Unif(unit circle), r_m ~ Unif([0, R]).
    All features share the SAME global center x_c and radius R.
    """
    def __init__(self, M, rng, gamma=6.0, xc=None, R=None):
        self.M = int(M)
        self.gamma = float(gamma)
        # --- global center x_c ---
        if xc is None:
            xc = np.array([ (x_min+x_max)/2.0, (y_min+y_max)/2.0 ])  # (0.25, 0.5)
        self.xc = xc.astype(float)

        # --- global radius R that slightly covers Ω ---
        if R is None:
            corners = np.array([[x_min,y_min],[x_max,y_min],[x_min,y_max],[x_max,y_max]])
            R_min = np.max(np.linalg.norm(corners - self.xc[None,:], axis=1))
            R = 1.05 * R_min   # 5% slack to "slightly cover"
        self.R = float(R)

        # directions a_m uniformly on unit circle
        th = rng.uniform(0, 2*np.pi, size=M)
        self.a  = np.stack([np.cos(th), np.sin(th)], axis=1)  # (M,2), |a_m|=1

        # scalar shifts r_m ~ Unif([0, R])
        self.r  = rng.uniform(0.0, self.R, size=M)            # (M,)

    def clone_structure(self):     #创建一个与当前对象具有相同结构但独立的新对象
        obj = object.__new__(RandFeatStrict)
        obj.M = self.M; obj.gamma = self.gamma
        obj.xc = self.xc.copy(); obj.R = self.R
        obj.a  = self.a.copy();  obj.r  = self.r.copy()
        return obj

    def set_gamma(self, gamma): self.gamma = float(gamma)

    def _g(self, Z):    #本函数表示的是tanh里面的部分
        # g_nm = γ * ( a_m^T (Z - xc) + r_m )
        Zrel = Z - self.xc[None,:]                         # (N,2)
        proj = Zrel @ self.a.T                              # (N,M)
        return self.gamma * (proj + self.r[None,:])        # (N,M)

    def Phi(self, Z):
        return np.tanh(self._g(Z))                         # (N,M)

    def dPhi_dx(self, Z):
        g = self._g(Z); sech2 = 1.0 - np.tanh(g)**2
        ax = self.a[:,0][None,:]
        return sech2 * (self.gamma * ax)

    def dPhi_dy(self, Z):
        g = self._g(Z); sech2 = 1.0 - np.tanh(g)**2
        ay = self.a[:,1][None,:]
        return sech2 * (self.gamma * ay)

    def laplacian(self, Z):
        g = self._g(Z); tg = np.tanh(g); sech2 = 1.0 - tg**2
        # since |a_m|=1 for every m: Δφ = -2 tanh(g) sech^2(g) * γ^2
        return -2.0 * tg * sech2 * (self.gamma**2)

# ------------------ Assemble LS (momentum + divergence + BC) ------------------
# 下面函数是为了picard iteration而设计的

def assemble_picard_ls(U_prev, Ru, Rp, Z_int, Z_bd,
                       w_int=1.0, w_div=1.0, w_bd=20.0, pin_pressure=True):   #Ru, Rp: velocity, pressure features
    """
    Unknowns: w = [a1(Mu), a2(Mu), bP(Mp), c1, c2]
    Rows:
      - Momentum residual on interior (2*Nint)
      - Divergence residual on interior (Nint)
      - Velocity Dirichlet on boundary (2*Nbd)
      - One pressure pin (1) [optional]
    """
    Nint = Z_int.shape[0]; Nbd = Z_bd.shape[0]
    Mu, Mp = Ru.M, Rp.M

    dUx = Ru.dPhi_dx(Z_int); dUy = Ru.dPhi_dy(Z_int); Lap_u = Ru.laplacian(Z_int)
    dPx = Rp.dPhi_dx(Z_int); dPy = Rp.dPhi_dy(Z_int)

    u1p = U_prev[:,0:1]; u2p = U_prev[:,1:2]

    # momentum residuals
    Ar1 = np.block([[u1p*dUx + u2p*dUy - nu*Lap_u, np.zeros((Nint,Mu)), dPx, np.zeros((Nint,1)), np.zeros((Nint,1))]])  #x方向动量方程
    Ar2 = np.block([[np.zeros((Nint,Mu)), u1p*dUx + u2p*dUy - nu*Lap_u, dPy, np.zeros((Nint,1)), np.zeros((Nint,1))]])  #y方向动量方程
    A_mom = np.vstack([Ar1, Ar2])
    b_mom = np.zeros((2*Nint,))

    # divergence residual
    A_div = np.block([[dUx, dUy, np.zeros((Nint, Mp)), np.zeros((Nint,1)), np.zeros((Nint,1))]])
    b_div = np.zeros((Nint,))

    # boundary Dirichlet for velocity (constants only appear here)
    Phi_bd = Ru.Phi(Z_bd); U_bd = u_exact(Z_bd); ones_bd = np.ones((Nbd,1))
    A_bd = np.block([
        [Phi_bd,               np.zeros((Nbd,Mu)), np.zeros((Nbd,Mp)), ones_bd,           np.zeros((Nbd,1))],
        [np.zeros((Nbd,Mu)),   Phi_bd,             np.zeros((Nbd,Mp)), np.zeros((Nbd,1)), ones_bd          ],
    ])
    b_bd = np.concatenate([U_bd[:,0], U_bd[:,1]], axis=0)

    # stack with weights
    A = np.vstack([w_int*A_mom, w_div*A_div, w_bd*A_bd])
    b = np.concatenate([w_int*b_mom, w_div*b_div, w_bd*b_bd], axis=0)

    # pressure gauge pin
    if pin_pressure:
        zref = Z_int[0:1,:]
        A_pin = np.concatenate([np.zeros((1,2*Mu)), Rp.Phi(zref), np.zeros((1,2))], axis=1)
        b_pin = p_exact(zref).ravel()
        A = np.vstack([A, A_pin]); b = np.concatenate([b, b_pin], axis=0)

    return A, b

def solve_picard_and_eta(Ru, Rp, Z_int, Z_bd, K=10, ridge=1e-10,
                         w_int=1.0, w_div=1.0, w_bd=20.0):
    """Run K Picard steps; return (a1,a2,bP,c1,c2), eta=||A w - b||^2 (last step)."""
    Mu, Mp = Ru.M, Rp.M
    U_prev = np.zeros((Z_int.shape[0], 2))
    a1=a2=bP=c1=c2=None; eta=None

    for _ in range(K):
        A, b = assemble_picard_ls(U_prev, Ru, Rp, Z_int, Z_bd,
                                  w_int=w_int, w_div=w_div, w_bd=w_bd, pin_pressure=True)
        ATA = A.T @ A + ridge*np.eye(A.shape[1]); ATb = A.T @ b
        w = np.linalg.solve(ATA, ATb)

        a1 = w[0:Mu]; a2 = w[Mu:2*Mu]; bP = w[2*Mu:2*Mu+Mp]    #这里这样子写是因为之前都排列在一起了，因此注意索引
        c1 = w[2*Mu+Mp]; c2 = w[2*Mu+Mp+1]

        Phi_int = Ru.Phi(Z_int)
        U_prev = np.stack([Phi_int@a1 + c1, Phi_int@a2 + c2], axis=1)

        r = A @ w - b; eta = float(r @ r)

    return (a1,a2,bP,c1,c2), eta

# -------------------- Golden-section search --------------------
def golden_section_search(obj_fn, a, b, tol=1e-2, maxit=50):
    gr = (math.sqrt(5)-1)/2
    c=b-gr*(b-a); d=a+gr*(b-a)
    fc=obj_fn(c); fd=obj_fn(d)
    it=0
    while (b-a)>tol and it<maxit:
        if fc<fd:
            b,d,fd=d,c,fc
            c=b-gr*(b-a); fc=obj_fn(c)
        else:
            a,c,fc=c,d,fd
            d=a+gr*(b-a); fd=obj_fn(d)
        it+=1
    x=(a+b)/2; fx=obj_fn(x)
    return x,fx

# ------------------------------ Main ------------------------------
def main():
    rng = np.random.default_rng(42)
    outdir = "nonlinear operator output photos"
    os.makedirs(outdir, exist_ok=True)

    # feature sizes
    Mu, Mp = 600, 300

    # build base feature structures (only γ tuned)
    Ru_base = RandFeatStrict(Mu, rng, gamma=6.0)   # velocity (global xc,R inside)
    Rp_base = RandFeatStrict(Mp, rng, gamma=6.0)   # pressure (same xc,R idea)

    # collocation (paper setting)
    Z_int, _, _ = uniform_grid_2d(50,50, x_min,x_max, y_min,y_max)   # 2500
    Z_bd = boundary_points_2d(50, x_min,x_max, y_min,y_max)          # 200

    # weights
    w_int, w_div, w_bd = 1.0, 1.0, 1.0

    def eta_of(gu, gp, K_search=3):
        Ru = Ru_base.clone_structure(); Ru.set_gamma(gu)
        Rp = Rp_base.clone_structure(); Rp.set_gamma(gp)
        _, eta = solve_picard_and_eta(Ru, Rp, Z_int, Z_bd, K=K_search,
                                      w_int=w_int, w_div=w_div, w_bd=w_bd)
        return eta

    # search intervals
    gu_lo, gu_hi = 1e-1 , 10.0
    gp_lo, gp_hi = 1e-1 , 10.0

    # coordinate-wise GS: γ_u then γ_p
    gp_curr = 6.0
    gamma_u_opt, eta_u = golden_section_search(lambda gu: eta_of(gu, gp_curr, K_search=3),
                                               gu_lo, gu_hi, tol=1e-2, maxit=50)
    print(f"[GS] gamma_u* = {gamma_u_opt:.3f},  eta = {eta_u:.6e}")

    gamma_p_opt, eta_p = golden_section_search(lambda gp: eta_of(gamma_u_opt, gp, K_search=3),
                                               gp_lo, gp_hi, tol=1e-2, maxit=50)
    print(f"[GS] gamma_p* = {gamma_p_opt:.3f},  eta = {eta_p:.6e}")

    # final training with more Picard steps
    Ru = Ru_base.clone_structure(); Ru.set_gamma(gamma_u_opt)
    Rp = Rp_base.clone_structure(); Rp.set_gamma(gamma_p_opt)
    (a1,a2,bP,c1,c2), eta_final = solve_picard_and_eta(Ru, Rp, Z_int, Z_bd, K=10,
                                                       w_int=w_int, w_div=w_div, w_bd=w_bd)
    print(f"[Train] final eta = {eta_final:.6e}")

    # test on random points
    Z_te = random_in_box(10_000, x_min,x_max, y_min,y_max, rng)
    U_true = u_exact(Z_te)
    Phi_te = Ru.Phi(Z_te)
    U_pred = np.stack([Phi_te@a1 + c1, Phi_te@a2 + c2], axis=1)
    mse_u = float(np.mean((U_pred - U_true)**2))
    print(f"[Test] Velocity MSE: {mse_u:.6e}")    #这里输出u的mse

    # (optional) pressure MSE (gauge pinned)
    P_pred = Rp.Phi(Z_te) @ bP
    P_true = p_exact(Z_te)
    mse_p = float(np.mean((P_pred - P_true)**2))
    print(f"[Test] Pressure MSE: {mse_p:.6e}")   #这里输出p的mse

    # visualization
    Zg, _, _ = uniform_grid_2d(150,150, x_min,x_max, y_min,y_max)
    Ug_pred = np.stack([Ru.Phi(Zg)@a1 + c1, Ru.Phi(Zg)@a2 + c2], axis=1).reshape(150,150,2)
    Ug_true = u_exact(Zg).reshape(150,150,2)
    Err = np.linalg.norm(Ug_pred - Ug_true, axis=2)

    plt.figure(figsize=(12,4.6))
    plt.subplot(1,3,1)
    plt.imshow(Ug_true[:,:,0], origin='lower', extent=[x_min,x_max,y_min,y_max], aspect='auto')
    plt.title("u1 (exact)"); plt.colorbar(fraction=0.046, pad=0.04)
    plt.subplot(1,3,2)
    plt.imshow(Ug_pred[:,:,0], origin='lower', extent=[x_min,x_max,y_min,y_max], aspect='auto')
    plt.title("u1 (pred)");  plt.colorbar(fraction=0.046, pad=0.04)
    plt.subplot(1,3,3)
    plt.imshow(Err, origin='lower', extent=[x_min,x_max,y_min,y_max], aspect='auto')
    plt.title("||u_pred - u_exact||");  plt.colorbar(fraction=0.046, pad=0.04)
    plt.tight_layout(); plt.savefig(os.path.join(outdir,"heatmaps_u1_err_div_cover.png"), dpi=160); plt.close()

    print(f"[Done] Results saved in: {outdir}")

if __name__ == "__main__":
    main()


[GS] gamma_u* = 4.432,  eta = 8.103853e-02
[GS] gamma_p* = 4.896,  eta = 6.677131e-02
[Train] final eta = 3.506586e-04
[Test] Velocity MSE: 6.561382e-08
[Test] Pressure MSE: 5.586671e-06
[Done] Results saved in: nonlinear operator output photos
